In [1]:
import pandas as pd
import os

data_dir = '../../data/panama-electricity-load-forecasting/processed'

if os.path.exists(os.path.join(data_dir, 'xPatch_concated_data.csv')): # and os.path.exists(os.path.join(data_dir, 'xPatch_test.csv')):
    pass
else:
    train_df = pd.read_parquet(os.path.join(data_dir, 'train.parquet')).drop(columns=['Holiday_ID', 'holiday', 'school'])
    test_df = pd.read_parquet(os.path.join(data_dir, 'test.parquet')).drop(columns=['Holiday_ID', 'holiday', 'school'])

    train_df = train_df.rename(columns = {'datetime': 'date'})
    test_df = test_df.rename(columns = {'datetime': 'date'})

    # test_df.to_csv(os.path.join(data_dir, 'xPatch_train.csv'), index=False)
    # train_df.to_csv(os.path.join(data_dir, 'xPatch_test.csv'), index=False)

    df = pd.concat([train_df, test_df]).reset_index(drop=True)

    all_columns = list(df.columns)
    all_columns.remove('nat_demand')

    df = df[all_columns + ['nat_demand']]
    df.to_csv(os.path.join(data_dir, 'xPatch_concated_data.csv'), index=False)

In [2]:
import optuna
import torch
import numpy as np

from xPatch_repo.exp.exp_main import Exp_Main

In [3]:
import sys
from datetime import datetime, timedelta
sys.path.append('..')

from utils.calculate_metrics import calculate_sklearn_metrics, get_quantile_from_median
from utils.plotting import plot_single_forecast

def objective(trial):
    data_dir = '../../data/panama-electricity-load-forecasting/processed'
    
    class Args:
        def __init__(self):
            self.seq_len = trial.suggest_categorical('seq_len', [24 * 4 * 2, 24 * 4 * 3, 24 * 4 * 4, 24 * 4 * 5])
            self.label_len = 24 * 4
            self.pred_len = 24 * 4
            self.is_training = 1
            self.model_id = 'optuna_test'
            self.model = 'xPatch'
            self.data = 'custom'
            self.root_path = data_dir
            self.data_path = "xPatch_concated_data.csv"
            self.features = 'MS'
            self.target = 'nat_demand'
            self.freq = 'h'
            self.scale = True
            self.timeenc = 0
            self.train_only = False
            self.enc_in = 13
            self.embed = 'timeF'
            self.patch_len = trial.suggest_int('patch_len', 8, 24, 4)
            self.stride = trial.suggest_int('stride', 4, 12, 2)
            self.padding_patch = 'end'
            #self.ma_type = trial.suggest_categorical('ma_type', ['ema', 'dema'])
            self.ma_type = 'ema'
            self.alpha = trial.suggest_float('alpha', 0.1, 0.5, step=0.1)
            #self.beta = trial.suggest_float('beta', 0.1, 0.5, step=0.1)
            self.beta = 0.3
            self.batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])
            self.train_epochs = 100
            self.patience = 5
            self.learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-3, log=True)
            self.loss = 'mse'
            self.lradj = 'type1'
            self.use_amp = False
            self.revin = 1
            self.use_gpu = True if torch.cuda.is_available() else False
            self.gpu = 0
            self.use_multi_gpu = False
            self.devices = '0'
            self.test_flop = False
            self.checkpoints = './checkpoints/'
            self.des = 'optuna_trial'
            self.itr = 1
            self.num_workers = 10
            self.train_perc = 0.9
            self.test_perc = 0.05
    
    args = Args()
    experiment = Exp_Main(args)

    ii = 0
    setting = '{}_{}_{}_ft{}_sl{}_ll{}_pl{}_{}_{}'.format(
                args.model_id,
                args.model,
                args.data,
                args.features,
                args.seq_len,
                args.label_len,
                args.pred_len,
                args.des, ii)

    model = experiment.train(setting)

    test_dataset, test_loader = experiment._get_data(flag='val')

    all_preds = []
    all_true = []

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    for i in range(0, len(test_dataset), args.pred_len):
        batch_x, batch_y, _, _ = test_dataset[i]
        batch_x = torch.tensor(batch_x).unsqueeze(0).float().to(device)
        batch_y = torch.tensor(batch_y).unsqueeze(0).float().to(device)

        model.eval()
        with torch.no_grad():
            preds = model(batch_x)

        y_pred = test_dataset.inverse_transform(preds.detach().squeeze(0).cpu().numpy())[..., -1:].flatten()
        y_true = test_dataset.inverse_transform(batch_y.detach().squeeze(0).cpu().numpy())[:preds.shape[1], -1:].flatten()

        all_preds.append(y_pred)
        all_true.append(y_true)

    y_pred = np.concatenate(all_preds)
    y_true = np.concatenate(all_true)

    start_date = datetime(2020, 1, 1)
    n_hours = y_pred.shape[0]

    date_range = [start_date + timedelta(hours=i) for i in range(n_hours)]

    results_df = pd.DataFrame()

    results_df['data'] = date_range
    results_df['y_pred'] = y_pred
    results_df['y_true'] = y_true

    df_for_metrics = results_df.copy().rename(columns={'y_pred': '0.5', 'y_true': 'target'})
    df_for_metrics['0.1'] = get_quantile_from_median(df_for_metrics['0.5'].values, target_quantile=0.1)
    df_for_metrics['0.9'] = get_quantile_from_median(df_for_metrics['0.5'].values, target_quantile=0.9)
    sklearn_metrics = calculate_sklearn_metrics(df_for_metrics)

    return sklearn_metrics['MSE']

In [4]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

best_params = trial.params

[I 2025-04-23 13:43:51,972] A new study created in memory with name: no-name-f5394530-2612-4fdb-a2d9-9f9b048e99ff


Use GPU: cuda:0
train 39588
val 2132
test 2130
	iters: 100, epoch: 1 | loss: 0.1155685
	speed: 0.0132s/iter; left time: 1633.2576s
	iters: 200, epoch: 1 | loss: 0.0873987
	speed: 0.0039s/iter; left time: 477.2134s
	iters: 300, epoch: 1 | loss: 0.0828633
	speed: 0.0037s/iter; left time: 462.2027s
	iters: 400, epoch: 1 | loss: 0.0792768
	speed: 0.0038s/iter; left time: 472.4144s
	iters: 500, epoch: 1 | loss: 0.0715570
	speed: 0.0037s/iter; left time: 461.5667s
	iters: 600, epoch: 1 | loss: 0.0708247
	speed: 0.0038s/iter; left time: 469.7357s
	iters: 700, epoch: 1 | loss: 0.0774524
	speed: 0.0047s/iter; left time: 578.3901s
	iters: 800, epoch: 1 | loss: 0.0661352
	speed: 0.0040s/iter; left time: 492.1831s
	iters: 900, epoch: 1 | loss: 0.0693105
	speed: 0.0040s/iter; left time: 494.0300s
	iters: 1000, epoch: 1 | loss: 0.0764594
	speed: 0.0039s/iter; left time: 480.2709s
	iters: 1100, epoch: 1 | loss: 0.0667723
	speed: 0.0042s/iter; left time: 510.9218s
	iters: 1200, epoch: 1 | loss: 0.0636

[I 2025-04-23 13:45:50,601] Trial 0 finished with value: 17084.267578125 and parameters: {'seq_len': 384, 'patch_len': 12, 'stride': 12, 'alpha': 0.5, 'batch_size': 32, 'learning_rate': 7.834685388883715e-05}. Best is trial 0 with value: 17084.267578125.


Epoch: 18, Steps: 1237 | Train Loss: 0.0599529 Vali Loss: 0.0645514 Test Loss: 0.2676977
EarlyStopping counter: 5 out of 5
Early stopping
val 2132
Use GPU: cuda:0
train 39492
val 2132
test 2130
	iters: 100, epoch: 1 | loss: 0.1826811
	speed: 0.0093s/iter; left time: 1152.7820s
	iters: 200, epoch: 1 | loss: 0.1466753
	speed: 0.0063s/iter; left time: 781.6252s
	iters: 300, epoch: 1 | loss: 0.1150823
	speed: 0.0060s/iter; left time: 737.2540s
	iters: 400, epoch: 1 | loss: 0.1057340
	speed: 0.0060s/iter; left time: 737.4886s
	iters: 500, epoch: 1 | loss: 0.0979333
	speed: 0.0060s/iter; left time: 734.7098s
	iters: 600, epoch: 1 | loss: 0.0944745
	speed: 0.0060s/iter; left time: 737.6497s
	iters: 700, epoch: 1 | loss: 0.0848772
	speed: 0.0060s/iter; left time: 735.5497s
	iters: 800, epoch: 1 | loss: 0.0808812
	speed: 0.0060s/iter; left time: 730.9986s
	iters: 900, epoch: 1 | loss: 0.0852702
	speed: 0.0059s/iter; left time: 725.4270s
	iters: 1000, epoch: 1 | loss: 0.0842274
	speed: 0.0060s/i

[I 2025-04-23 13:48:23,346] Trial 1 finished with value: 16526.873046875 and parameters: {'seq_len': 480, 'patch_len': 16, 'stride': 12, 'alpha': 0.5, 'batch_size': 32, 'learning_rate': 1.7161881141142703e-05}. Best is trial 1 with value: 16526.873046875.


...

Epoch: 14, Steps: 1237 | Train Loss: 0.0595668 Vali Loss: 0.0641471 Test Loss: 0.2656223
EarlyStopping counter: 5 out of 5
Early stopping
val 2132
Use GPU: cuda:0
train 39588
val 2132
test 2130
	iters: 100, epoch: 1 | loss: 0.1901951
	speed: 0.0070s/iter; left time: 859.3192s
	iters: 200, epoch: 1 | loss: 0.1588760
	speed: 0.0038s/iter; left time: 465.7526s
	iters: 300, epoch: 1 | loss: 0.1405143
	speed: 0.0038s/iter; left time: 466.1760s
	iters: 400, epoch: 1 | loss: 0.1222549
	speed: 0.0038s/iter; left time: 464.8402s
	iters: 500, epoch: 1 | loss: 0.1168064
	speed: 0.0037s/iter; left time: 451.9716s
	iters: 600, epoch: 1 | loss: 0.1011477
	speed: 0.0038s/iter; left time: 464.8255s
	iters: 700, epoch: 1 | loss: 0.1061670
	speed: 0.0038s/iter; left time: 467.6714s
	iters: 800, epoch: 1 | loss: 0.0999266
	speed: 0.0038s/iter; left time: 467.6226s
	iters: 900, epoch: 1 | loss: 0.1014066
	speed: 0.0038s/iter; left time: 462.6408s
	iters: 1000, epoch: 1 | loss: 0.0961775
	speed: 0.0038s/it

[I 2025-04-23 15:33:45,786] Trial 48 finished with value: 15523.3544921875 and parameters: {'seq_len': 384, 'patch_len': 12, 'stride': 10, 'alpha': 0.5, 'batch_size': 32, 'learning_rate': 1.3754830324963765e-05}. Best is trial 44 with value: 15025.2333984375.


Epoch: 24, Steps: 1237 | Train Loss: 0.0696955 Vali Loss: 0.0723207 Test Loss: 0.2879554
EarlyStopping counter: 5 out of 5
Early stopping
val 2132
Use GPU: cuda:0
train 39588
val 2132
test 2130
	iters: 100, epoch: 1 | loss: 0.1621233
	speed: 0.0078s/iter; left time: 481.4971s
	iters: 200, epoch: 1 | loss: 0.1254898
	speed: 0.0046s/iter; left time: 285.5230s
	iters: 300, epoch: 1 | loss: 0.1097093
	speed: 0.0046s/iter; left time: 283.4970s
	iters: 400, epoch: 1 | loss: 0.0943358
	speed: 0.0047s/iter; left time: 285.6368s
	iters: 500, epoch: 1 | loss: 0.0888705
	speed: 0.0046s/iter; left time: 284.1810s
	iters: 600, epoch: 1 | loss: 0.0909209
	speed: 0.0047s/iter; left time: 286.3777s
Epoch: 1 cost time: 3.2662453651428223
Epoch: 1, Steps: 618 | Train Loss: 0.1184474 Vali Loss: 0.0840853 Test Loss: 0.3328884
Validation loss decreased (inf --> 0.084085).  Saving model ...
Updating learning rate to 3.362796022585753e-05
	iters: 100, epoch: 2 | loss: 0.0805242
	speed: 0.0209s/iter; left tim

[I 2025-04-23 15:34:52,231] Trial 49 finished with value: 16033.5283203125 and parameters: {'seq_len': 384, 'patch_len': 8, 'stride': 12, 'alpha': 0.5, 'batch_size': 64, 'learning_rate': 3.362796022585753e-05}. Best is trial 44 with value: 15025.2333984375.


Epoch: 15, Steps: 618 | Train Loss: 0.0672323 Vali Loss: 0.0703450 Test Loss: 0.2846329
EarlyStopping counter: 5 out of 5
Early stopping
val 2132
Best trial:
  Value: 15025.2333984375
  Params: 
    seq_len: 384
    patch_len: 8
    stride: 12
    alpha: 0.5
    batch_size: 32
    learning_rate: 1.3005841984812764e-05


In [5]:
print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

best_params = trial.params

Best trial:
  Value: 15025.2333984375
  Params: 
    seq_len: 384
    patch_len: 8
    stride: 12
    alpha: 0.5
    batch_size: 32
    learning_rate: 1.3005841984812764e-05


In [11]:
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_param_importances

fig = plot_optimization_history(study)
fig.show()

fig = plot_param_importances(study)
fig.show()